### Import libraries

In [ ]:
!pip install demoji

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import unicodedata as uni
import demoji
from tqdm import tqdm
import spacy
import requests
import gensim.corpora as corpora
from gensim.models import Phrases
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Import file

In [ ]:
url = 'https://raw.githubusercontent.com/morayod/M_S_C_D_S/main/reviews.csv'
df = pd.read_csv(url, encoding="utf-8")
df.head(10)

Understand Data

In [ ]:
df.info()

### Remove URLS

In [ ]:
import re


def remove_url(text):
    text = re.sub(r"http\S+", "", text)
    return text


### Clean emojis

In [ ]:
def handle_emoji(string):
    emojis = demoji.findall(string)

    for emoji in emojis:
        string = string.replace(emoji, " " + emojis[emoji].split(":")[0])

    return string


### Tokenise words

In [ ]:
def word_tokeniser(text):
    text = text.lower()
    text = text.split()

    return text


sample = "Hi Everyone I really love the playlists on this app."
print(sample)
print(word_tokeniser(sample))


## Here I created a custom list of stopwords including sentiment words, and the name of the app as well as other unimportant words

In [ ]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#Adding in extra stopwords related to sentiment and unneeded characters
custom_stopwords = ["spotify", "i", "I", "app", "happy", "sad", "work", "hi", "fun", "idk", "lol", "ive", "trash", "god", "guys", "dont", "worst", "days", "right", "thanks", "lots", "week", "nice", "kind", "day", "ca", "garbage", "today", "sucks", "awesome", "everytime", "bit", "thank", "easy", "things", "keeps", "people", "lot", "thing", "way", "times", "im", "angry", "ve", "music", "great", "don", "want", "good", "really", "love", "hate", "songs", "song", "like", "just"]
en_stopwords = set(stopwords.words('english'))
all_stopwords = set(custom_stopwords).union(set(ENGLISH_STOP_WORDS).union(en_stopwords))
print(f"Stop Words in English : \n{ all_stopwords}")

### Remove Stopwords function

In [ ]:
def remove_stopwords(text):
    text = [word for word in text if word not in all_stopwords]
    return text


print(f"Before removing stopwords : {word_tokeniser(sample)}")
print(f"After removing stopwords : {remove_stopwords(word_tokeniser(sample))}")

### Lemmatisation step below

In [ ]:
sp = spacy.load("en_core_web_sm")
def lemmatisation(text):

    text = " ".join(text)
    token = sp(text)

    text = [word.lemma_ for word in token]
    return text


print(f"Before Lemmatisation : {word_tokeniser(sample)}")
print(f"After Lemmatisation : {lemmatisation(word_tokeniser(sample))}")

### Define pre-processing function

In [ ]:
def preprocessing(text):
    
    text = remove_url(text) 
    text = uni.normalize('NFKD', text)
    text = handle_emoji(text)
    text = text.lower() 
    text = re.sub(r'[^\w\s]', '', text)
    text = word_tokeniser(text)
    text = lemmatisation(text)
    text = remove_stopwords(text)

    text = " ".join(text)
    return text

### Apply pre-processing

In [ ]:
tqdm.pandas()

df['clean_review'] = df['Review'].progress_apply(lambda x: preprocessing(x))


### See what cleaned review looks like

In [ ]:
df.head()

### Tokenise clean review

In [ ]:
tqdm.pandas()

df['tokenised_review'] = df['clean_review'].progress_map(word_tokeniser)
data_words = df['tokenised_review'].values.tolist()

df.head(10)



### Create list of all words

In [ ]:
data_words = df['tokenised_review'].values.tolist()
len(data_words)
all_words = df['Review'].values.tolist()

### Create a dictionary and corpus

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
corpus_bigram = [id2word.doc2bow(doc) for doc in data_words]
# View
print(corpus[:1][0][:30])



### Example preprocessing 

In [ ]:
tokenised_corpus = [[id2word[word_id] for (word_id, count) in doc] for doc in corpus]

processed_text = preprocessing("I love spotify so much it's the best thing every and I'm impressed by the audio quality")
print(processed_text)

### LDA model building for topic modelling

In [ ]:
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from pprint import pprint

# number of topics
num_topics = 10 
# Build LDA model
lda_model = LdaMulticore(corpus=corpus, id2word=id2word, num_topics=num_topics, iterations=400)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


Although topics aren't really well defined, I've manually chosen these labels from the topics for the further analysis:

advert,
subscription_or_cost,
song_selection,
user_experience,
comparison,
audio_quality,
support,
podcasts,
connectivity,
podcast,
recommendation,
playlists

Below I will visualise the clusters from the LDA analysis

In [ ]:
from sklearn.manifold import TSNE
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label
from bokeh.palettes import Category10
output_notebook()

# Get Document-Topic Distributions
doc_topic_dists = np.zeros((len(corpus), num_topics))
for doc_num, doc_topics in enumerate(lda_model[corpus_bigram]):
    for topic, prob in doc_topics:
        doc_topic_dists[doc_num, topic] = prob
print(doc_topic_dists.shape)


# Use t-SNE to reduce dimensionality
tsne_model = TSNE(n_components=2, verbose=1, random_state=0, angle=.99, init='pca')
tsne_lda = tsne_model.fit_transform(doc_topic_dists)

# Visualising using Bokeh
n_topics = lda_model.num_topics
colormap = np.array(Category10[10])
_lda_keys = np.array(doc_topic_dists).argmax(axis=1).tolist()

# Finding the centroid of each topic
_mean_topic_vectors = []
for t in range(n_topics):
    if colormap[_lda_keys].tolist().count(colormap[t]) > 0:
        _mean_topic_vectors.append(tsne_lda[colormap[_lda_keys] == colormap[t]].mean(axis=0))
        
top_3_words_lda = []
for t in range(n_topics):
    topic_words = lda_model.show_topic(t, 3)
    words_for_topic = [word for word, score in topic_words]
    top_3_words_lda.append(", ".join(words_for_topic))
    



The mean sigma is not great but this is fine for our type of analysis

In [ ]:
plot = figure(title="t-SNE Clustering of {} LDA Topics".format(n_topics), width=700, height=700)
plot.scatter(x=tsne_lda[:,0], y=tsne_lda[:,1], color=colormap[_lda_keys])
for t in range(n_topics):
    label = Label(x=_mean_topic_vectors[t][0], y=_mean_topic_vectors[t][1], 
                  text=top_3_words_lda[t], text_color=colormap[t])
    plot.add_layout(label)

show(plot, notebook_handle=True)

Clusters are not well defined but this is fine as we were able to pre-define topics for our analysis below

Next to train the FastText model

In [ ]:
input_url = 'https://raw.githubusercontent.com/morayod/M_S_C_D_S/main/assistant_label_responses.txt'

response = requests.get(input_url)

file_content = response.text

# Display the first 500 characters
    
print(file_content[:500])  
    
# Calculate the word count
word_count = len(file_content.split())
print(f"Word count: {word_count}")


In [ ]:
lines = response.text.split('\n')
len(lines)


### Split training and validation data and train

In [ ]:
import fasttext
train_data = "\n".join(lines[:1735])  # Adjust the number of lines you want for training
valid_data = "\n".join(lines[1735:])  # Adjust the number of lines for validation
# Split the data into train and validation sets
with open("spotify.train", "w", encoding="utf-8") as train_file:
        train_file.write(train_data)
        
with open("spotify.valid", "w", encoding="utf-8") as valid_file:
        valid_file.write(valid_data)


model = fasttext.train_supervised(input="spotify.train", lr=0.73, epoch=67, wordNgrams=1, bucket=200000, dim=50, loss='ova', seed=7077)
model.save_model("model_spotify_final.bin") 

Test in a sample review not from the training set

In [ ]:
model.predict("I hate the ads despite paying so much, and audio just sounds tinny", k=12)

The model correctly has high probability for the above example for the aspects "advert", audio_quality" and "subscription_or_cost" 

## Calculate the optimal precision and recall picking the best threshold

In [ ]:
thresholds = np.linspace(1, 0, 100)  
precisions = []
recalls = []
f1_scores = []
num_labels = 5

for threshold in thresholds:
    _, precision, recall = model.test("spotify.valid", k=num_labels, threshold=threshold)
    
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(2 * (precision * recall) / (precision + recall))

# Plotting
plt.figure(figsize=(10,7))
plt.plot(recalls, precisions, color='blue')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()

# Highlight the maximum F1-score point
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
plt.scatter(recalls[optimal_idx], precisions[optimal_idx], color='red')
plt.annotate(f'Threshold:{optimal_threshold:.2f}', (recalls[optimal_idx], precisions[optimal_idx]))

plt.show()


Calculate precision and recall using optimal threshold. 

In [ ]:
threshold = optimal_threshold
model.test("spotify.valid", k=num_labels, threshold=threshold)

It's not a bad precision or recall - we can adjust the threshold hgiher for increased precision but there will be a tradeoff for recall
Define an aspect extraction function

In [ ]:
def get_aspects(text):
    try:
        return model.predict(text, k=num_labels, threshold=threshold)
    except:
        return 0

In [ ]:
tqdm.pandas(desc="Calculating Similarities")
df['label'] = df['Review'].progress_apply(lambda text: get_aspects(text))

In [ ]:
pd.set_option('display.max_colwidth', 100)
df.tail(5)

In [ ]:
from nltk.tokenize import sent_tokenize
from transformers import pipeline

sentiment_pipeline = pipeline('sentiment-analysis')

def split_on_conjunctions(text):
    splits = re.split(r'\b(but|or|because|so|although|though|while)\b|\s*,\s*', text)
    return [s.strip() for s in splits if s]

def get_sentiment(text):
    result = sentiment_pipeline(text)
    return result[0]['label']

def get_absa(review):
    sentences = sent_tokenize(review)
    label_sentiments = {}

    for sentence in sentences:
        segments = split_on_conjunctions(sentence)
        for segment in segments:
            sentiment = get_sentiment(segment)
            if sentiment == 'POSITIVE':
                sentiment_label = "Positive"
            elif sentiment == 'NEGATIVE':
                sentiment_label = "Negative"
            else:
                sentiment_label = "Neutral"

            labels, probabilities = model.predict(segment, threshold=threshold, k=num_labels)

            for i, label in enumerate(labels):
                stripped_label = label.replace("__label__", "")
                if stripped_label not in label_sentiments or probabilities[i] > label_sentiments[stripped_label][1]:
                    label_sentiments[stripped_label] = (sentiment_label, probabilities[i])

    # Remtving the probabilities and keeping only the sentiments for final output
    final_sentiments = {label: sentiment[0] for label, sentiment in label_sentiments.items()}
    return final_sentiments
    

review = ("Easiest and most convenient way for a student to stream music," 
" but the app is plagued with issues, like pausing randomly on my Samsung" 
" S8 *edit* Issues still persist on Samsung S21. I'm paying far too much for the "
"premium I wish it cost less but I kind of enjoy the listening to the podcasts "
"available. Please help  "
"the customer support team is lacking as they don't communicate nor reply and I absolutely hate how slow it is to connect my device on wifi and all the buzzing sounds are annoying")

get_absa(review)

In [ ]:
datafile = 'https://raw.githubusercontent.com/morayod/M_S_C_D_S/main/randomly%20shuffled_reviews_only.csv'

In [ ]:
test_df = pd.read_csv(datafile, encoding="utf-8")
test_df_100 = test_df.head(100)
tqdm.pandas(desc="Calculating Similarities")
test_df_100['absa'] = test_df_100['Review'].progress_apply(lambda text: get_absa(text))

In [ ]:
test_df_100

A great job on my sample reviews!

In [ ]:
test_df_100.to_csv("lda_model_output.csv", encoding='utf-8', index=False)